In [50]:
import arcpy

MAP_NAME = "Map" #Please add custom project information here
CORE_LAYER_NAME = "Symbology_Source"
TERRAIN_FIELD_NAME = "Type" #This field name causes issues. It could be worthwhile to resolve in the future.
TERRAIN_TYPES = {
    "Dense Woods",
    "Woodlands",
    "Mesic Forest",
    "Floodplain Forest",
    "Flatwoods",
    "Wooded Dune",
    "Oak Savanna",
    "Shrubland",
    "Agricultural",
    "Residential",
    "Tallgrass Prairie"
    "Mesic Prairie",
    "Wet Prairie",
    "Wetland/Marsh",
    "Fen",
    "Swamp",
    "Sand",
    "Water",
    "Industrial"
}

### Get Core Layer Address ###
print("Retrieving Symbology Layer Object...")
aprx = arcpy.mp.ArcGISProject("CURRENT") #retrieve project object
results = aprx.listMaps(MAP_NAME) #retrieve map object
assert(len(results) == 1) #sanity check
mp = results[0]

results = mp.listLayers(CORE_LAYER_NAME) #retrieve layer object
assert(len(results) == 1) #sanity check
sourceLyr = results[0]
print("\t", sourceLyr.dataSource)
print("Complete", "\n")

### Retrieve Trail Map Layers ###
print("Determining Target Layers...")
targetLyrs = []
for lyr in mp.listLayers(): #iterate through all layers
    if lyr.name in ["World Topographic Map", "World Hillshade"]: continue #These layers has a field of the name "Type". Attempting to access that data will crash the script.
    if lyr.name == sourceLyr.name: continue #Do not include the source file in the target list
    print("\t", lyr.name)
    # Filter 1: Only look for layers with a matching field name
    lyrFields = [field.name for field in arcpy.ListFields(lyr)]
    if TERRAIN_FIELD_NAME in lyrFields:
        #Filter 2: Only look for layers with matching attribute values
        with arcpy.da.SearchCursor(lyr, "Type") as cursor:
            enteries = {row[0] for row in cursor}
            if len(enteries & TERRAIN_TYPES) > 0:
                targetLyrs.append(lyr)
                
print("Complete", "\n")

### Copy Symbology ###
print("Copying Symbology..")

for targetLyr in targetLyrs:
    print("\t", targetLyr)
    arcpy.management.ApplySymbologyFromLayer(targetLyr, sourceLyr)
print("Complete", "\n")

Retrieving Symbology Layer Object...
	 C:\Users\sevmy\OneDrive\Documents\ArcGIS\Packages\Symbology_Source_8ae039\commondata\default.gdb\Symbology_Source
Complete 

Determining Target Layers...
	 Propogation_Test
Complete 

Copying Symbology..
Complete 

